In [37]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import paths_cpt 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc, RocCurveDisplay
import os

print(os.getcwd())
seed = 22

df = pd.read_parquet(paths_cpt.PATH_TO_PARQUET)
print(df.shape)
df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\volte\OneDrive\Dokumente\usecase_cpt\CPT_usecase\modeling
(1222711, 19)


,sondering_id,index,pkey_sondering,sondeernummer,x,y,start_sondering_mtaw,diepte_sondering_tot,diepte,diepte_mtaw,qc,fs,qtn,rf,fr,icn,sbt,ksbt,lithostrat_id
0,314,2593,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.6,13.66,1.17,0.035,35.894004,2.991453,3.058371,2.564340,5.0,1.434000e-07,Quartair
1,314,2594,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.7,13.56,1.57,0.033,42.562319,2.101911,2.138968,2.406724,5.0,4.321000e-07,Quartair
2,314,2595,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.8,13.46,1.43,0.036,38.536991,2.517483,2.569226,2.491219,5.0,2.392000e-07,Quartair
3,314,2596,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.9,13.36,0.50,0.024,15.678501,4.800000,5.111166,2.982185,3.0,7.700000e-09,Quartair
4,314,2597,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,2.0,13.26,1.33,0.023,33.203119,1.729323,1.772110,2.440158,5.0,3.419000e-07,Quartair


In [38]:
print("Unique lithostrats : ", df.lithostrat_id.unique())
print("Count NAs : ", df.isna().sum())

Unique lithostrats :  ['Quartair' 'Mont_Panisel' 'Aalbeke' None 'Mons_en_Pevele'
 'Quartair + Mont_Panisel' 'Gentbrugge' 'Egem' 'Maldegem' 'Brussel'
 'Onbekend' 'Ursel' 'Asse' 'Wemmel' 'Lede' 'Bolderberg' 'Tielt'
 'Merelbeke' 'Kwatrecht' 'Antropogeen' 'Vlierzele'
 'Schelde Groep + Mons_en_Pevele' 'Veldwezelt en Gembloux' 'Diest'
 'Orchies' 'Tertiair' 'Egemkapel' 'Sint_Huibrechts_Hern'
 'Quartair_Holoceen' 'Onbekend + Mont_Panisel']
Count NAs :  sondering_id                  0
index                         0
pkey_sondering           469344
sondeernummer                 0
x                             0
y                             0
start_sondering_mtaw          0
diepte_sondering_tot          0
diepte                        0
diepte_mtaw                   0
qc                            0
fs                            0
qtn                           2
rf                            0
fr                            0
icn                         762
sbt                         762
ksbt   

In [39]:
target_set = df[(df["lithostrat_id"].isna()) | (df["lithostrat_id"]=="None") | df["lithostrat_id"].str.contains('Onbekend')]
train_set = df[~((df["lithostrat_id"].isna()) | (df["lithostrat_id"]=="None") | df["lithostrat_id"].str.contains('Onbekend'))]
print("rows in train and target are : ", train_set.shape[0], " ; ", target_set.shape[0], ",",\
       target_set.shape[0] + train_set.shape[0], " = ", df.shape[0])

rows in train and target are :  196012  ;  1026699 , 1222711  =  1222711


In [40]:
target_set.lithostrat_id.unique()
train_set.lithostrat_id.unique()

array(['Quartair', 'Mont_Panisel', 'Aalbeke', 'Mons_en_Pevele',
       'Quartair + Mont_Panisel', 'Gentbrugge', 'Egem', 'Maldegem',
       'Brussel', 'Ursel', 'Asse', 'Wemmel', 'Lede', 'Bolderberg',
       'Tielt', 'Merelbeke', 'Kwatrecht', 'Antropogeen', 'Vlierzele',
       'Schelde Groep + Mons_en_Pevele', 'Veldwezelt en Gembloux',
       'Diest', 'Orchies', 'Tertiair', 'Egemkapel',
       'Sint_Huibrechts_Hern', 'Quartair_Holoceen'], dtype=object)

In [41]:
train_set.dropna(inplace=True)
y = train_set.lithostrat_id
X = train_set.drop(columns = ["lithostrat_id"])
X.drop(columns=["pkey_sondering", "sondeernummer", "x", "y", "sondering_id", "index"], inplace=True)
X.head()

C:\Users\volte\AppData\Local\Temp\ipykernel_50072\1033929061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.dropna(inplace=True)


,start_sondering_mtaw,diepte_sondering_tot,diepte,diepte_mtaw,qc,fs,qtn,rf,fr,icn,sbt,ksbt
0,15.26,25.4,1.6,13.66,1.17,0.035,35.894004,2.991453,3.058371,2.564340,5.0,1.434000e-07
1,15.26,25.4,1.7,13.56,1.57,0.033,42.562319,2.101911,2.138968,2.406724,5.0,4.321000e-07
2,15.26,25.4,1.8,13.46,1.43,0.036,38.536991,2.517483,2.569226,2.491219,5.0,2.392000e-07
3,15.26,25.4,1.9,13.36,0.50,0.024,15.678501,4.800000,5.111166,2.982185,3.0,7.700000e-09
4,15.26,25.4,2.0,13.26,1.33,0.023,33.203119,1.729323,1.772110,2.440158,5.0,3.419000e-07


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_test.head())

(111933, 12) (37312, 12) (111933,) (37312,)
515035           Brussel
534331           Brussel
467323              Lede
20863     Mons_en_Pevele
213930           Brussel
Name: lithostrat_id, dtype: object


In [43]:
#One vs rest, one model per category
model1 = LogisticRegression(random_state=seed, multi_class="ovr", class_weight="balanced").fit(X_train, y_train)

c:\Users\volte\Documents\gcp_practice\GCP_functions_and_scripts\tf_on_gcp_experiments\.env\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\volte\Documents\gcp_practice\GCP_functions_and_scripts\tf_on_gcp_experiments\.env\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\volte\Documents\gcp_practice\GCP_functions_and_s

In [44]:
y_pred = model1.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Overall accuracy:", accuracy)

classes = y_test.unique()
matrix = confusion_matrix(y_test, y_pred, labels=classes)

#This is apparently how you get per calss accuracy: 
# https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class
granular_acc  = matrix.diagonal()/matrix.sum(axis=1)

df_summary = pd.DataFrame({
    "class": classes,
    "accuracy": granular_acc
})
print("avg accuracy : ", df_summary.accuracy.mean())
print("Individual accuracies : ")
df_summary

Overall accuracy: 0.4193021012006861
avg accuracy :  0.5039778837105164
Individual accuracies : 


,class,accuracy
0,Brussel,0.425615
1,Lede,0.430713
2,Mons_en_Pevele,0.211250
3,Quartair,0.315120
4,Sint_Huibrechts_Hern,0.653395
5,Aalbeke,0.511290
6,Wemmel,0.470361
7,Diest,0.602101
8,Mont_Panisel,0.492754
9,Quartair_Holoceen,0.280172


In [45]:
print(classification_report(y_test, y_pred))

                                precision    recall  f1-score   support

                       Aalbeke       0.50      0.51      0.51       620
                   Antropogeen       0.02      0.10      0.04       273
                          Asse       0.39      0.45      0.42      1234
                    Bolderberg       0.16      0.18      0.17       525
                       Brussel       0.65      0.43      0.52      6137
                         Diest       0.51      0.60      0.55      2189
                          Egem       0.01      0.67      0.01         6
                     Egemkapel       0.04      0.92      0.08        64
                    Gentbrugge       0.16      0.77      0.27       196
                     Kwatrecht       0.34      0.38      0.36       696
                          Lede       0.62      0.43      0.51      4936
                      Maldegem       0.06      0.36      0.10       242
                     Merelbeke       0.13      0.72      0.23  

## Adjusting for multicollinearity and VIF as verified in R

In [46]:

selected_vars = ['diepte', 'fs', 'qtn', 'fr', 'icn', 'sbt', 'ksbt']
X_train_selected = X_train[selected_vars]
X_test_selected = X_test[selected_vars]
model2 = LogisticRegression(random_state=seed, class_weight="balanced").fit(X_train_selected, y_train)

c:\Users\volte\Documents\gcp_practice\GCP_functions_and_scripts\tf_on_gcp_experiments\.env\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [47]:
y_pred = model2.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print("Overall accuracy:", accuracy)

classes = y_test.unique()
matrix = confusion_matrix(y_test, y_pred, labels=classes)

#This is apparently how you get per calss accuracy: 
# https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class
granular_acc  = matrix.diagonal()/matrix.sum(axis=1)

df_summary = pd.DataFrame({
    "class": classes,
    "accuracy": granular_acc
})

print("avg accuracy : ", df_summary.accuracy.mean())
print("Individual accuracies : ")
df_summary

Overall accuracy: 0.18428387650085765
avg accuracy :  0.31608059866175325
Individual accuracies : 


,class,accuracy
0,Brussel,0.148444
1,Lede,0.091167
2,Mons_en_Pevele,0.342500
3,Quartair,0.061051
4,Sint_Huibrechts_Hern,0.015824
5,Aalbeke,0.551613
6,Wemmel,0.496993
7,Diest,0.227958
8,Mont_Panisel,0.432913
9,Quartair_Holoceen,0.655172


In [48]:
print(classification_report(y_test, y_pred))

                                precision    recall  f1-score   support

                       Aalbeke       0.36      0.55      0.44       620
                   Antropogeen       0.04      0.47      0.07       273
                          Asse       0.51      0.29      0.37      1234
                    Bolderberg       0.04      0.05      0.05       525
                       Brussel       0.46      0.15      0.22      6137
                         Diest       0.44      0.23      0.30      2189
                          Egem       0.00      0.17      0.00         6
                     Egemkapel       0.13      0.80      0.22        64
                    Gentbrugge       0.03      0.12      0.05       196
                     Kwatrecht       0.27      0.20      0.23       696
                          Lede       0.52      0.09      0.16      4936
                      Maldegem       0.04      0.07      0.05       242
                     Merelbeke       0.03      0.83      0.06  

## Random forest approach for nonlinear model

In [49]:
from sklearn.ensemble import RandomForestClassifier

model3 = RandomForestClassifier(
    n_estimators=200,
    random_state=seed,
    class_weight='balanced'
)

model3.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=200,
                       random_state=22)

In [50]:
y_pred = model3.predict(X_test)

In [51]:
y_pred = model3.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Overall accuracy:", accuracy)

classes = y_test.unique()
matrix = confusion_matrix(y_test, y_pred, labels=classes)

#This is apparently how you get per calss accuracy: 
# https://stackoverflow.com/questions/39770376/scikit-learn-get-accuracy-scores-for-each-class
granular_acc  = matrix.diagonal()/matrix.sum(axis=1)

df_summary = pd.DataFrame({
    "class": classes,
    "accuracy": granular_acc
})

print("avg accuracy : ", df_summary.accuracy.mean())
print("Individual accuracies : ")
df_summary

Overall accuracy: 0.9939161663807891
avg accuracy :  0.9851770821946341
Individual accuracies : 


,class,accuracy
0,Brussel,0.996741
1,Lede,0.996961
2,Mons_en_Pevele,0.980000
3,Quartair,0.996647
4,Sint_Huibrechts_Hern,0.996937
5,Aalbeke,0.972581
6,Wemmel,0.990550
7,Diest,0.999086
8,Mont_Panisel,0.992987
9,Quartair_Holoceen,1.000000


In [52]:
print(classification_report(y_test, y_pred))

                                precision    recall  f1-score   support

                       Aalbeke       0.98      0.97      0.98       620
                   Antropogeen       1.00      0.99      0.99       273
                          Asse       0.98      0.99      0.98      1234
                    Bolderberg       1.00      0.98      0.99       525
                       Brussel       1.00      1.00      1.00      6137
                         Diest       1.00      1.00      1.00      2189
                          Egem       1.00      1.00      1.00         6
                     Egemkapel       1.00      0.98      0.99        64
                    Gentbrugge       1.00      0.99      0.99       196
                     Kwatrecht       0.99      0.99      0.99       696
                          Lede       1.00      1.00      1.00      4936
                      Maldegem       1.00      0.99      1.00       242
                     Merelbeke       0.97      0.90      0.93  

In [53]:
X_train.columns

Index(['start_sondering_mtaw', 'diepte_sondering_tot', 'diepte', 'diepte_mtaw',
       'qc', 'fs', 'qtn', 'rf', 'fr', 'icn', 'sbt', 'ksbt'],
      dtype='object')

In [54]:
#Trying out statsmodels library
import numpy as np

y_codes = pd.Categorical(y_train).codes
X_train_enc = sm.add_constant(X_train)

#Multinomial logit, slightly different than OvR
mnlogit = sm.MNLogit(y_codes, X_train_enc)
mnlogit_fit = mnlogit.fit()

c:\Users\volte\Documents\gcp_practice\GCP_functions_and_scripts\tf_on_gcp_experiments\.env\lib\site-packages\statsmodels\discrete\discrete_model.py:3027: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
c:\Users\volte\Documents\gcp_practice\GCP_functions_and_scripts\tf_on_gcp_experiments\.env\lib\site-packages\statsmodels\discrete\discrete_model.py:3028: RuntimeWarning: invalid value encountered in divide
  return eXB/eXB.sum(1)[:,None]


Optimization terminated successfully.
         Current function value: nan
         Iterations 3
